# Association Analysis: Basic Concepts and Algorithms

In [ ]:
pkgs <- sort(c('tidyverse', 'arules', 'arulesViz', 'mlbench'))

lapply(pkgs, function(pkg) {
  if (system.file(package = pkg) == '') install.packages(pkg)
})

all_pkgs <- union(all_pkgs, pkgs)

**Packages used for this chapter:** `r format_pkgs(pkgs)`

You can read the free sample chapter from the textbook [@Tan2005]:
[Chapter 5. Association Analysis: Basic Concepts and
Algorithms](https://www-users.cs.umn.edu/~kumar001/dmbook/ch5_association_analysis.pdf)

## The arules Package

Association rule mining in R is implemented in the package `arules`.

In [ ]:
library(tidyverse)
library(arules)
library(arulesViz)

For information about the `arules` package try: `help(package="arules")`
and `vignette("arules")` (also available at
[CRAN](http://cran.r-project.org/web/packages/arules/vignettes/arules.pdf))

arules uses the S4 object system to implement classes and methods.
Standard R objects use the [S3 object
system](http://adv-r.had.co.nz/S3.html) which do not use formal class
definitions and are usually implemented as a list with a class
attribute. `arules` and many other R packages use the [S4 object
system](http://adv-r.had.co.nz/S4.html) which is based on formal class
definitions with member variables and methods (similar to
object-oriented programming languages like Java and C++). Some important
differences of using S4 objects compared to the usual S3 objects are:

-   coercion (casting): `as(from, "class_name")`
-   help for classes: `class? class_name`

## Transactions

### Create Transactions

We will use the Zoo dataset from `mlbench`.

In [ ]:
data(Zoo, package = "mlbench")
head(Zoo)

The data in the data.frame need to be converted into a set of
transactions where each row represents a transaction and each column is
translated into items. This is done using the constructor
`transactions()`. For the Zoo data set this means that we consider
animals as transactions and the different traits (features) will become
items that each animal has. For example the animal *antelope* has the
item *hair* in its transaction.

In [ ]:
trans <- transactions(Zoo)

The conversion gives a warning because only discrete features (`factor`
and `logical`) can be directly translated into items. Continuous
features need to be discretized first.

What is column 13?

In [ ]:
summary(Zoo[13])
ggplot(Zoo, aes(legs)) + geom_bar()
table(Zoo$legs)

Possible solution: Make legs into has/does not have legs

In [ ]:
Zoo_has_legs <- Zoo %>% mutate(legs = legs > 0)
ggplot(Zoo_has_legs, aes(legs)) + geom_bar()
table(Zoo_has_legs$legs)

**Alternatives:**

-   use each unique value as an item:

In [ ]:
Zoo_unique_leg_values <- Zoo %>% mutate(legs = factor(legs))
head(Zoo_unique_leg_values$legs)

-   discretize (see
    [`? discretize`](https://www.rdocumentation.org/packages/arules/topics/discretize)
    and [discretization in the code for Chapter
    2](chap2.html#discretize-features)):

In [ ]:
Zoo_discretized_legs <- Zoo %>% mutate(
  legs = discretize(legs, breaks = 2, method="interval")
)
table(Zoo_discretized_legs$legs)

Convert data into a set of transactions

In [ ]:
trans <- transactions(Zoo_has_legs)
trans

### Inspect Transactions

In [ ]:
summary(trans)

Look at created items. They are still called column names since the
transactions are actually stored as a large sparse logical matrix (see
below).

In [ ]:
colnames(trans)

Compare with the original features (column names) from Zoo

In [ ]:
colnames(Zoo)

Look at a (first) few transactions as a matrix. 1 indicates the presence
of an item.

In [ ]:
as(trans, "matrix")[1:3,]

Look at the transactions as sets of items

In [ ]:
inspect(trans[1:3])

Plot the binary matrix. Dark dots represent 1s.

In [ ]:
image(trans)

Look at the relative frequency (=support) of items in the data set. Here
we look at the 10 most frequent items.

In [ ]:
itemFrequencyPlot(trans,topN = 20)

ggplot(
  tibble(
    Support = sort(itemFrequency(trans, type = "absolute"), decreasing = TRUE),
    Item = seq_len(ncol(trans))
  ), aes(x = Item, y = Support)) + geom_line()

**Alternative encoding:** Also create items for FALSE (use factor)

In [ ]:
sapply(Zoo_has_legs, class)
Zoo_factors <- Zoo_has_legs %>% mutate_if(is.logical, factor)
sapply(Zoo_factors, class)
summary(Zoo_factors)

trans_factors <- transactions(Zoo_factors)
trans_factors

itemFrequencyPlot(trans_factors, topN = 20)

## Select transactions that contain a certain item
trans_insects <- trans_factors[trans %in% "type=insect"]
trans_insects
inspect(trans_insects)

### Vertical Layout (Transaction ID Lists)

The default layout for transactions is horizontal layout (i.e. each
transaction is a row). The vertical layout represents transaction data
as a list of transaction IDs for each item (= transaction ID lists).

In [ ]:
vertical <- as(trans, "tidLists")
as(vertical, "matrix")[1:10, 1:5]

## Frequent Itemsets

### Mine Frequent Itemsets

For this dataset we have already a huge number of possible itemsets

In [ ]:
2^ncol(trans)

Find frequent itemsets (target="frequent") with the default settings.

In [ ]:
its <- apriori(trans, parameter=list(target = "frequent"))
its

Default minimum support is .1 (10%). **Note:** We use here a very small
data set. For larger datasets the default minimum support might be to
low and you may run out of memory. You probably want to start out with a
higher minimum support like .5 (50%) and then work your way down.

In [ ]:
5/nrow(trans)

In order to find itemsets that effect 5 animals I need to go down to a
support of about 5%.

In [ ]:
its <- apriori(trans, parameter=list(target = "frequent", support = 0.05))
its

Sort by support

In [ ]:
its <- sort(its, by = "support")
inspect(head(its, n = 10))

Look at frequent itemsets with many items (set breaks manually since
Automatically chosen breaks look bad)

In [ ]:
ggplot(tibble(`Itemset Size` = factor(size(its))), aes(`Itemset Size`)) + geom_bar()
inspect(its[size(its) > 8])

### Concise Representation of Itemsets

Find maximal frequent itemsets (no superset if frequent)

In [ ]:
its_max <- its[is.maximal(its)]
its_max
inspect(head(its_max, by = "support"))

Find closed frequent itemsets (no superset if frequent)

In [ ]:
its_closed <- its[is.closed(its)]
its_closed
inspect(head(its_closed, by = "support"))

counts <- c(
  frequent=length(its),
  closed=length(its_closed),
  maximal=length(its_max)
)

ggplot(as_tibble(counts, rownames = "Itemsets"),
  aes(Itemsets, counts)) + geom_bar(stat = "identity")

## Association Rules

### Mine Association Rules

We use the APRIORI algorithm (see
[`? apriori`](https://www.rdocumentation.org/packages/arules/topics/apriori))

In [ ]:
rules <- apriori(trans, parameter = list(support = 0.05, confidence = 0.9))
length(rules)

inspect(head(rules))
quality(head(rules))

Look at rules with highest lift

In [ ]:
rules <- sort(rules, by = "lift")
inspect(head(rules, n = 10))

Create rules using the alternative encoding (with "FALSE" item)

In [ ]:
r <- apriori(trans_factors)
r
print(object.size(r), unit = "Mb")

inspect(r[1:10])
inspect(head(r, n = 10, by = "lift"))

### Calculate Additional Interest Measures

In [ ]:
interestMeasure(rules[1:10], measure = c("phi", "gini"),
  trans = trans)

Add measures to the rules

In [ ]:
quality(rules) <- cbind(quality(rules),
  interestMeasure(rules, measure = c("phi", "gini"),
    trans = trans))

Find rules which score high for Phi correlation

In [ ]:
inspect(head(rules, by = "phi"))

### Mine Using Templates

Sometimes it is beneficial to specify what items should be where in the
rule. For apriori we can use the parameter appearance to specify this
(see
[`? APappearance`](https://www.rdocumentation.org/packages/arules/topics/APappearance)).
In the following we restrict rules to an animal `type` in the RHS and
any item in the LHS.

In [ ]:
type <- grep("type=", itemLabels(trans), value = TRUE)
type

rules_type <- apriori(trans, appearance= list(rhs = type))

inspect(head(sort(rules_type, by = "lift")))

Saving rules as a CSV-file to be opened with Excel or other tools.

`write(rules, file = "rules.csv", quote = TRUE)`

## Association Rule Visualization

In [ ]:
library(arulesViz)

Default scatterplot

In [ ]:
plot(rules)

Note that some jitter (randomly move points) was added to show how many
rules have the same confidence and support value. Without jitter:

In [ ]:
plot(rules, control = list(jitter = 0))

plot(rules, shading = "order")
##plot(rules, interactive = TRUE)

Grouped plot

In [ ]:
plot(rules, method = "grouped")
##plot(rules, method = "grouped", engine = "interactive")

As a graph

In [ ]:
plot(rules, method = "graph")
plot(head(rules, by = "phi", n = 100), method = "graph")

## Interactive Visualizations

We will use the association rules mined from the Iris dataset for the
following examples.

In [ ]:
data(iris)
summary(iris)

Convert the data to transactions. Note that the features are numeric and
need to be discretized. The conversion automatically applies
frequency-based discretization with 3 classes to each numeric feature
(with a warning).

In [ ]:
iris_trans <- transactions(iris)
inspect(head(iris_trans))

Next, we mine association rules.

In [ ]:
rules <- apriori(iris_trans, parameter = list(support = 0.1, confidence = 0.8))
rules

### Interactive Inspect With Sorting, Filtering and Paging

In [ ]:
inspectDT(rules)

### Scatter Plot

Plot rules as a scatter plot using an interactive html widget. To avoid
overplotting, jitter is added automatically. Set `jitter = 0` to disable
jitter. Hovering over rules shows rule information. *Note:*
plotly/javascript does not do well with too many points, so plot selects
the top 1000 rules with a warning if more rules are supplied.

In [ ]:
plot(rules, engine = "html")

### Matrix Visualization

Plot rules as a matrix using an interactive html widget.

In [ ]:
plot(rules, method = "matrix", engine = "html") 

### Visualization as Graph

Plot rules as a graph using an interactive html widget. *Note:* the used
javascript library does not do well with too many graph nodes, so plot
selects the top 100 rules only (with a warning).

In [ ]:
plot(rules, method = "graph", engine = "html")

### Interactive Rule Explorer

You can specify a rule set or a dataset. To explore rules that can be
mined from iris, use: `ruleExplorer(iris)`

The rule explorer creates an interactive Shiny application that can be
used locally or deployed on a server for sharing. A deployed version of
the ruleExplorer is available
[here](https://mhahsler-apps.shinyapps.io/ruleExplorer_demo/) (using
[shinyapps.io](https://www.shinyapps.io/)).